<a href="https://colab.research.google.com/github/elimeyer1/ML_4105/blob/main/HW5_q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/My Drive/4105/Housing.csv'
housing = pd.read_csv(file_path)
housing.head()


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [ ]:
varlist =  ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']

def redefine(x):
    return x.map({'yes': 1, 'no': 0})

In [ ]:
housing[varlist] = housing[varlist].apply(redefine)
housing.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,furnished
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,furnished
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,semi-furnished
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,furnished
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,furnished


In [ ]:
x = housing[['area', 'bedrooms', 'bathrooms', 'stories','mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'parking']].values
y = housing['price'].values

In [ ]:
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
class LinearRegressionModel(nn.Module):
    def __init__(self, input_size):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_size, 1)

    def forward(self, x):
        return self.linear(x)

In [ ]:
def train_model(x_train, y_train, x_val, y_val, optimizer, learning_rate, num_epochs=5000):
    input_size = x_train.shape[1]
    model = LinearRegressionModel(input_size)
    criterion = nn.MSELoss()
    train_dataset = TensorDataset(x_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    optimizer = optimizer(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for batch_x, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        if (epoch + 1) % 500 == 0:
            model.eval()
            with torch.no_grad():
                val_predictions = model(x_val)
                val_loss = criterion(val_predictions, y_val)
        return model

In [ ]:
learning_rates = [0.1, 0.01, 0.001, 0.0001]
optimizers = [optim.Adam]

In [ ]:
best_model = None
best_loss = float('inf')
for optimizer_type in optimizers:
    for lr in learning_rates:
        print(f"Training with {optimizer_type.__name__} optimizer and learning rate {lr}")
        model = train_model(x_train, y_train, x_val, y_val, optimizer_type, lr)
        with torch.no_grad():
            val_predictions = model(x_val)
            val_loss = nn.MSELoss()(val_predictions, y_val)
            print(f'Validation Loss: {val_loss}')
        if val_loss < best_loss:
            best_model = model
            best_loss = val_loss

Training with Adam optimizer and learning rate 0.1


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([20])) that is different to the input size (torch.Size([20, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([109])) that is different to the input size (torch.Size([109, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(inpu

Validation Loss: 30063403728896.0
Training with Adam optimizer and learning rate 0.01
Validation Loss: 30127096332288.0
Training with Adam optimizer and learning rate 0.001
Validation Loss: 30129474502656.0
Training with Adam optimizer and learning rate 0.0001
Validation Loss: 30130233671680.0
